<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Supervised Learning Model Comparison

---

### Let us begin...

Recall the `data science process`.
   1. Define the problem.
   2. Gather the data.
   3. Explore the data.
   4. Model the data.
   5. Evaluate the model.
   6. Answer the problem.

In this lab, we're going to focus mostly on creating (and then comparing) many regression and classification models. Thus, we'll define the problem and gather the data for you.
Most of the questions requiring a written response can be written in 2-3 sentences.

### Step 1: Define the problem.

You are a data scientist with a financial services company. Specifically, you want to leverage data in order to identify potential customers.

If you are unfamiliar with "401(k)s" or "IRAs," these are two types of retirement accounts. Very broadly speaking:
- You can put money for retirement into both of these accounts.
- The money in these accounts gets invested and hopefully has a lot more money in it when you retire.
- These are a little different from regular bank accounts in that there are certain tax benefits to these accounts. Also, employers frequently match money that you put into a 401k.
- If you want to learn more about them, check out [this site](https://www.nerdwallet.com/article/ira-vs-401k-retirement-accounts).

We will tackle one regression problem and one classification problem today.
- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.

Check out the data dictionary [here](http://fmwww.bc.edu/ec-p/data/wooldridge2k/401KSUBS.DES).

#### NOTE: When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable. 

#### When predicting `e401k`, you may use the entire dataframe if you wish.

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier, AdaBoostRegressor, AdaBoostClassifier
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import svm

### Step 2: Gather the data.

##### 1. Read in the data.

In [2]:
df = pd.read_csv('401ksubs.csv')

In [3]:
df.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809


##### 2. What are 2-3 other variables that, if available, would be helpful to have?

In [ ]:
# Two other variables
#tax bracket and spouse income

##### 3. Suppose a peer recommended putting `race` into your model in order to better predict who to target when advertising IRAs and 401(k)s. Why would this be an unethical decision?

In [ ]:
#due to race discrimination thus introducing bias, and regulatory issues as some states it is illegal to ask for race information.

## Step 3: Explore the data.

##### 4. When attempting to predict income, which feature(s) would we reasonably not use? Why?

In [ ]:
#gender, age, marital status due to ethical reasons that introduce bias and discrimination.

##### 5. What two variables have already been created for us through feature engineering? Come up with a hypothesis as to why subject-matter experts may have done this.
> This need not be a "statistical hypothesis." Just brainstorm why SMEs (Subject Matter Experts) might have done this!

In [ ]:
#the income is squared to deal with non-linearity, improving model fit (stabilize variance, handle interaction effects between low income and high income) 

##### 6. Looking at the data dictionary, one variable description appears to be an error. What is this error, and what do you think the correct value would be?

In [ ]:
#the first age column is not squared, it is just the age. 'agesq' is already squared. 

## Step 4: Model the data. (Part 1: Regression Problem)

Recall:
- Problem: What features best predict one's income?
- When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable.

##### 7. List all modeling tactics we've learned that could be used to solve a regression problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific regression problem and explain why or why not.

In [7]:
df.columns

#List all models since Week 6: knn, logitic regression, decision trees(regressor), random forest, XGBoost 
#Since  the y is numerical, it is a regression problem, assuming the LINE assumptions. 
#the goal is to predict income (y-target variable) by using the given features (predictors in X). 

Index(['e401k', 'inc', 'marr', 'male', 'age', 'fsize', 'nettfa', 'p401k',
       'pira', 'incsq', 'agesq'],
      dtype='object')

##### 8. Regardless of your answer to number 7, fit at least one of each of the following models to attempt to solve the regression problem above:
    - a multiple linear regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend setting a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

In [9]:
df.isnull().sum()

e401k     0
inc       0
marr      0
male      0
age       0
fsize     0
nettfa    0
p401k     0
pira      0
incsq     0
agesq     0
dtype: int64

In [21]:
df.describe().round(3)

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
count,9275.000,9275.000,9275.000,9275.000,9275.00,9275.000,9275.000,9275.000,9275.000,9275.000,9275.000
mean,0.392,39.255,0.629,0.204,41.08,2.885,19.072,0.276,0.254,2121.192,1793.653
std,0.488,24.090,0.483,0.403,10.30,1.526,63.964,0.447,0.436,3001.469,895.649
min,0.000,10.008,0.000,0.000,25.00,1.000,-502.302,0.000,0.000,100.160,625.000
25%,0.000,21.660,0.000,0.000,33.00,2.000,-0.500,0.000,0.000,469.156,1089.000
50%,0.000,33.288,1.000,0.000,40.00,3.000,2.000,0.000,0.000,1108.091,1600.000
75%,1.000,50.160,1.000,0.000,48.00,4.000,18.450,1.000,1.000,2516.025,2304.000
max,1.000,199.041,1.000,1.000,64.00,13.000,1536.798,1.000,1.000,39617.320,4096.000


In [23]:
df.dtypes

e401k       int64
inc       float64
marr        int64
male        int64
age         int64
fsize       int64
nettfa    float64
p401k       int64
pira        int64
incsq     float64
agesq       int64
dtype: object

In [11]:
df['e401k'].value_counts(normalize=True).mul(100).round(2) # class size: 60% of people are not eligible, 40% eligible.

e401k
0    60.79
1    39.21
Name: proportion, dtype: float64

In [25]:
df.columns

Index(['e401k', 'inc', 'marr', 'male', 'age', 'fsize', 'nettfa', 'p401k',
       'pira', 'incsq', 'agesq'],
      dtype='object')

In [39]:
X = df[['marr', 'male', 'fsize', 'nettfa', 'agesq']]
y = df['incsq']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [29]:
X

,marr,male,fsize,nettfa,agesq
0,0,0,1,4.575,1600
1,0,1,1,154.000,1225
2,1,0,2,0.000,1936
3,1,1,2,21.800,1936
4,0,0,1,18.450,2809
...,...,...,...,...,...
9270,1,0,4,-1.200,1089
9271,0,1,3,2.000,1369
9272,1,0,3,-13.600,1089
9273,1,0,3,3.550,3249


In [56]:
sc = StandardScaler()

In [58]:
sc.fit(X_train)

StandardScaler()

In [60]:
X_train_sc = sc.transform(X_train)

In [64]:
X_test_sc = sc.transform(X_test)

## Model 1 SVM 

In [52]:
svm_model = svm.SVR()

In [66]:
svm_model.fit(X_train_sc, y_train)

SVR()

In [68]:
cross_val_score(svm_model, X_train_sc, y_train).mean()

-0.06829671990880208

In [70]:
svm_model.score(X_train_sc, y_train)

-0.06097722225719848

In [72]:
svm_model.score(X_test_sc, y_test)

-0.060386906572619026

## Model 2 Ada Boost

In [75]:
ada_model = AdaBoostRegressor()

In [77]:
ada_model.fit(X_train_sc, y_train)

AdaBoostRegressor()

In [79]:
cross_val_score(ada_model, X_train_sc, y_train).mean()

-0.3965207036220127

In [81]:
ada_model.score(X_train_sc, y_train)

-0.2055313988649754

In [83]:
ada_model.score(X_test_sc, y_test)

-0.26781759979873665

## Model 3 Random Forest

In [85]:
rf_model = RandomForestRegressor()

In [87]:
rf_model.fit(X_train_sc, y_train)

RandomForestRegressor()

In [89]:
cross_val_score(rf_model, X_train_sc, y_train).mean()

0.23917812482692788

In [91]:
rf_model.score(X_train_sc, y_train)

0.8908656490198983

In [93]:
rf_model.score(X_test_sc, y_test)

0.1874546493309346

## Model 4 KNN

In [97]:
knn_model = KNeighborsRegressor()

In [99]:
knn_model.fit(X_train_sc, y_train)

KNeighborsRegressor()

In [101]:
cross_val_score(knn_model, X_train_sc, y_train).mean()

0.21144319227510655

In [103]:
knn_model.score(X_train_sc, y_train)

0.4808411805557512

In [105]:
knn_model.score(X_train_sc, y_train)

0.4808411805557512

##### 9. What is bootstrapping?

In [ ]:
# bootstrapping is to sample the training data with replacement, resulting in different training subsets for each tree. 
# each dataset slightly different from the others.
# By using bootstrapping, we create a more robust and reliable ensemble model that 
# uses the strengths of multiple decision trees (wisdom of the crowd), 
#reducing the risk of overfitting and improving generalization to new data

##### 10. What is the difference between a decision tree and a set of bagged decision trees? Be specific and precise!

In [ ]:
# Single Decision Tree: A single tree is trained on the entire dataset, leading to a specific decision boundary. 
# Simple, interpretable model but prone to overfitting.

# Bagged Decision Trees: Multiple trees (ensemble method) are trained on different subsets of the data, 
#resulting in a combined decision boundary that is generally more robust,
#less prone to overfitting and increases robustness, but with reduced interpretability.

##### 11. What is the difference between a set of bagged decision trees and a random forest? Be specific and precise!

In [ ]:
# Both are ensemble methods to improve the performance and robustness of decision trees by combining multiple trees. 
# Random forest: selects a random subset of features for each split within the trees to decorrelate the trees. 


#Bagged decision tree: All features are considered for each split within every tree.

##### 12. Why might a random forest be superior to a set of bagged decision trees?
> Hint: Consider the bias-variance tradeoff.

In [ ]:
# Random forest has less variance and decorrelation, and improved performance than bagging alone. Bagging mainly reduces just variance with ensembling.

## Step 5: Evaluate the model. (Part 1: Regression Problem)

##### 13. Using RMSE, evaluate each of the models you fit on both the training and testing data.

## SVM RMSE

In [107]:
# metrics is root_mean_squared_error # or we can use rmse = metrics.root_mean_squared_error(true,preds)
svm_preds = svm_model.predict(X_train_sc)

svm_rmse_train = sqrt(mean_squared_error(y_train, svm_preds))

print(svm_rmse_train)

3072.880584136111


In [110]:
svm_preds_test = svm_model.predict(X_test_sc)
svm_rmse_test = sqrt(mean_squared_error(y_test, svm_preds_test))
print(svm_rmse_test)

3145.6179172989428


## AdaBoost RMSE

In [118]:
ada_preds_train = ada_model.predict(X_train_sc)
ada_rmse_train = sqrt(mean_squared_error(y_train, ada_preds_train))
print(ada_rmse_train)

3275.5325265759147


In [120]:
ada_preds_test = ada_model.predict(X_test_sc)
ada_rmse_test = sqrt(mean_squared_error(y_test, ada_preds_test))
print(ada_rmse_test)

3439.5543774100465


## Random Forest RMSE

In [122]:
rf_preds_train = rf_model.predict(X_train_sc)
rf_rmse_train = sqrt(mean_squared_error(y_train, rf_preds_train))
print(rf_rmse_train)

985.538031666376


In [124]:
rf_preds_test = rf_model.predict(X_test_sc)
rf_rmse_test = sqrt(mean_squared_error(y_test, rf_preds_test))
print(rf_rmse_test)

2753.5793523183556


## kNN RMSE

In [128]:
knn_preds_train = knn_model.predict(X_train_sc)
knn_rmse_train = sqrt(mean_squared_error(y_train, knn_preds_train))
print(knn_rmse_train)

2149.5263402618393


In [130]:
knn_preds_test = knn_model.predict(X_test_sc)
knn_rmse_test = sqrt(mean_squared_error(y_test, knn_preds_test))
print(knn_rmse_test)

2696.482628491796


##### 14. Based on training RMSE and testing RMSE, is there evidence of overfitting in any of your models? Which ones?

In [ ]:
## If train RMSE score > test RMSE score, this shows that the model is overfitting. 
#SVM Model: not much overfit since train and test scores similar
#AdaBoost Model : not much overfit since train and test scores similar
#Random Forest Model : underfit as train is less than test score
#kNN Model : underfit as train is less than test score

##### 15. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

In [ ]:
#SVM had the least gap between train and test scores. 

##### 16. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

In [ ]:
# GridsearchCV for further hyperparameter tuning.
# add more features columns to prevent underfitting, such as dummifying different age groups

## Step 4: Model the data. (Part 2: Classification Problem)

Recall:
- Problem: Predict whether or not one is eligible for a 401k.
- When predicting `e401k`, you may use the entire dataframe if you wish.

##### 17. While you're allowed to use every variable in your dataframe, mention at least one disadvantage of using `p401k` in your model.

In [ ]:
# p401k =1 if participate in 401(k). The disadvantage is that it is mostly the same value (0 or 1) as e401k, thus the data is repeating.
#the percentage of different e401k and p401k is low (approx. 10%)

In [17]:
df['e401k'].value_counts(normalize=True).mul(100).round(2) 

e401k
0    60.79
1    39.21
Name: proportion, dtype: float64

In [15]:
df['p401k'].value_counts(normalize=True).mul(100).round(2)

p401k
0    72.38
1    27.62
Name: proportion, dtype: float64

In [13]:
df.head(20)

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809
5,0,15.000,1,0,60,3,0.000,0,0,225.0000,3600
6,0,37.155,1,0,49,5,3.483,0,1,1380.4940,2401
7,0,31.896,1,0,38,5,-2.100,0,0,1017.3550,1444
8,0,47.295,1,0,52,2,5.290,0,1,2236.8170,2704
9,1,29.100,0,1,45,1,29.600,0,1,846.8100,2025


##### 18. List all modeling tactics we've learned that could be used to solve a classification problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific classification problem and explain why or why not.

In [ ]:
#logistic regression (for binary classification), decision tree classifier, random forest classifier, gradient boosting classifier, XGBoost, SVM, KNN
#depending if the class is binary or multi class. 

##### 19. Regardless of your answer to number 18, fit at least one of each of the following models to attempt to solve the classification problem above:
    - a logistic regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend using a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

In [132]:
df.columns

Index(['e401k', 'inc', 'marr', 'male', 'age', 'fsize', 'nettfa', 'p401k',
       'pira', 'incsq', 'agesq'],
      dtype='object')

In [134]:
X = df[['marr', 'male', 'fsize', 'nettfa', 'pira', 'incsq', 'agesq']]
y = df['e401k']

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [138]:
ss = StandardScaler()

In [140]:
ss.fit(X_train)

StandardScaler()

In [142]:
X_train_sc = ss.transform(X_train)

In [144]:
X_test_sc = ss.transform(X_test)

## Logistic Regression

In [148]:
lr = LogisticRegression()

In [150]:
lr.fit(X_train_sc, y_train)

LogisticRegression()

In [152]:
cross_val_score(lr, X_train_sc, y_train).mean()

0.6364292826627663

In [154]:
lr.score(X_train_sc, y_train)

0.6365727429557216

In [158]:
lr.score(X_test_sc, y_test)

0.648124191461837

## Decision Tree

In [162]:
dt = DecisionTreeClassifier()

In [164]:
dt.fit(X_train_sc, y_train)

DecisionTreeClassifier()

In [166]:
cross_val_score(dt, X_train_sc, y_train).mean()

0.5904257253113199

In [168]:
dt.score(X_train_sc, y_train)

1.0

In [170]:
dt.score(X_test_sc, y_test)

0.5912031047865459

## Bagging

In [173]:
bag = BaggingClassifier()

In [175]:
bag.fit(X_train_sc, y_train)

BaggingClassifier()

In [177]:
cross_val_score(bag, X_train_sc, y_train).mean()

0.6472120652470313

In [179]:
bag.score(X_train_sc, y_train)

0.9772857964347326

In [181]:
bag.score(X_test_sc, y_test)

0.6511427339370418

## Step 5: Evaluate the model. (Part 2: Classfication Problem)

##### 20. Suppose our "positive" class is that someone is eligible for a 401(k). What are our false positives? What are our false negatives?

In [ ]:
#False Positives: predicts is eligible for a 401(k), but not eligible.

#False Negatives: predicts not eligible for a 401(k), but is eligible.

##### 21. In this specific case, would we rather minimize false positives or minimize false negatives? Defend your choice.

In [ ]:
# Since the cost is not symmetrical, we would want to minimize the one with the most loss in terms of the y target value.
# If we are an employer, I think the False Negatives have the most loss since there are regulations and compliance issues, and the employer loses on tax reduction
# whereas False Positives can be corrected and contributions simply returned back to the employee.

##### 22. Suppose we wanted to optimize for the answer you provided in problem 21. Which metric would we optimize in this case?

In [ ]:
# recall

##### 23. Suppose that instead of optimizing for the metric in problem 21, we wanted to balance our false positives and false negatives using `f1-score`. Why might [f1-score](https://en.wikipedia.org/wiki/F1_score) be an appropriate metric to use here?

In [ ]:
# the f1 score is the harmonic mean using both precision and recall, so neither false positives and false negatives are ignored.

#the F1 score is influenced more by the smaller value, highlighting models that balance both errors well.

# Single Value: Provides a single metric that simplifies model comparison and selection, especially when multiple models need to be evaluated.

##### 24. Using f1-score, evaluate each of the models you fit on both the training and testing data.

## Logistic regression f1-score

In [184]:
lr_preds_train = lr.predict(X_train_sc)

In [186]:
lr_f1_train = f1_score(y_train, lr_preds_train)

In [188]:
lr_f1_train

0.29777777777777775

In [190]:
lr_preds_test = lr.predict(X_test_sc)

In [192]:
lr_f1_test = f1_score(y_test, lr_preds_test)

In [194]:
lr_f1_test

0.31197301854974707

## Decision Tree f1-score

In [197]:
dt_preds_train = dt.predict(X_train_sc)

In [199]:
dt_f1_train = f1_score(y_train, dt_preds_train)

In [201]:
dt_f1_train

1.0

In [203]:
dt_preds_test = dt.predict(X_test_sc)

In [205]:
dt_f1_test = f1_score(y_test, dt_preds_test)

In [207]:
dt_f1_test

0.47739801543550164

## Bagging f1-score

In [210]:
bag_preds_train = bag.predict(X_train_sc)

In [212]:
bag_f1_train = f1_score(y_train, bag_preds_train)

In [214]:
bag_f1_train

0.9705443698732289

In [216]:
bag_preds_test = bag.predict(X_test_sc)

In [218]:
bag_f1_test = f1_score(y_test, bag_preds_test)

In [220]:
bag_f1_test

0.4817424727738629

##### 25. Based on training f1-score and testing f1-score, is there evidence of overfitting in any of your models? Which ones?

In [ ]:
# Decision tree and bagging had too much overfitting due to the higher train f1-score than test f1-score.

##### 26. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

In [ ]:
#logistic regression due to the train accuracy score is the most similar to the test accuracy score.

##### 27. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

In [ ]:
## Gridsearch, try using other classification models such as KNN, XGBoost, Random Forest, SVM models. 

## Step 6: Answer the problem.

##### BONUS: Briefly summarize your answers to the regression and classification problems. Be sure to include any limitations or hesitations in your answer.

- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.

In [ ]:
## Regression: the SVM model showed the best scores. Income 'inc' and Net financial assets 'nettfa' showed the best correlation to e401k, but the correlation was under 0.5, indicating a relatively weak correlation. 

## Classification: the logistic regression for classification had the best score, but it is not beating the baseline (60%) by much, thus the test predictions are mainly random,
# which indicates more investigation needed such as searching for another model, or more features. 

In [222]:
df.corr()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
e401k,1.000000,0.268178,0.080843,-0.027641,0.031526,0.012015,0.143950,0.769170,0.118643,0.206618,0.017526
inc,0.268178,1.000000,0.362008,-0.069871,0.105638,0.110170,0.376586,0.270833,0.364354,0.940161,0.087305
marr,0.080843,0.362008,1.000000,-0.363950,0.059047,0.564814,0.075039,0.085636,0.116925,0.280060,0.054500
male,-0.027641,-0.069871,-0.363950,1.000000,-0.120297,-0.320678,-0.018132,-0.024949,-0.036361,-0.053715,-0.116235
age,0.031526,0.105638,0.059047,-0.120297,1.000000,-0.030536,0.203906,0.025977,0.238557,0.097584,0.992619
fsize,0.012015,0.110170,0.564814,-0.320678,-0.030536,1.000000,-0.031506,0.014296,-0.043629,0.079570,-0.055924
nettfa,0.143950,0.376586,0.075039,-0.018132,0.203906,-0.031506,1.000000,0.187392,0.345917,0.407568,0.203703
p401k,0.769170,0.270833,0.085636,-0.024949,0.025977,0.014296,0.187392,1.000000,0.153033,0.222113,0.015740
pira,0.118643,0.364354,0.116925,-0.036361,0.238557,-0.043629,0.345917,0.153033,1.000000,0.322805,0.233543
incsq,0.206618,0.940161,0.280060,-0.053715,0.097584,0.079570,0.407568,0.222113,0.322805,1.000000,0.082991
